In [1]:
# default_exp playlists

# Playlists

<br>

### Imports

In [12]:
#exports
import pandas as pd
import numpy as np

import FEAutils as hlp
import matplotlib.pyplot as plt
import seaborn as sns

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import os
import requests

from spotifywatch.discography import retrieve_track_analysis, initialise_spotify_client

from ipypb import track

In [16]:
from IPython.display import JSON

<br>

### Spotipy Client Initialisation

In [3]:
import dotenv
_ = dotenv.load_dotenv('../.env')

In [13]:
sp = initialise_spotify_client()

sp

In [6]:
#exports
def setup_client_scopes():
    client_id = os.getenv('SPOTIFY_ID')
    
    spotify_auth_url = 'https://accounts.spotify.com/authorize'
    scopes = ['user-read-private', 'user-modify-playback-state', 'user-read-currently-playing']
    
    query_params = {
        'client_id' : client_id,
        'response_type' : 'code',
        'redirect_uri' : 'http://osuked.com',
        'scope' : ' '.join(scopes)
    }
    
    r = requests.get(spotify_auth_url, params=query_params)
    
    return r

In [7]:
r = setup_client_scopes()

r

<Response [200]>

<br>

### Playlist Data Retrieval

In [14]:
#exports
def get_album_art(track):    
    track_meta = dict()
    album_imgs = track['album']['images']
    
    if len(album_imgs) > 0:
        track_meta['album_art_url'] = album_imgs[0]['url']
        track_meta['album_art_height'] = album_imgs[0]['height']
        track_meta['album_art_width'] = album_imgs[0]['width']
        
    return track_meta

def get_album_data(track):
    track_meta = dict()
    
    track_meta['album_uri'] = track['album']['uri']
    track_meta['album_name'] = track['album']['name']
    track_meta['release_date'] = track['album']['release_date']
    
    return track_meta

def get_track_data(track):
    track_meta = dict()
    
    track_meta['artist_names'] = ', '.join([artist['name'] for artist in track['artists']])
    track_meta['artist_uris'] = ', '.join([artist['uri'] for artist in track['artists']])
    track_meta.update(track['external_ids'])
    
    track_meta['disc_number'] = track['disc_number']
    track_meta['duration_ms'] = track['duration_ms']
    track_meta['name'] = track['name']
    track_meta['track_uri'] = track['uri']
    track_meta['track_number'] = track['track_number']
    track_meta['popularity'] = track['popularity']
    
    return track_meta

def get_playlist_track_metadata(playlist_items):
    playlist_track_meta = []
    
    for playlist_entry in playlist_items['items']:
        track_meta = dict()
        
        track_meta['date_added_to_playlist'] = playlist_entry['added_at']
        track = playlist_entry['track']

        extractor_funcs = [get_album_art, get_album_data, get_track_data]

        for extractor_func in extractor_funcs:
            track_meta.update(extractor_func(track))
            
        playlist_track_meta += [track_meta]
        
    return playlist_track_meta

def retrieve_playlist_df(playlist_id='spotify:playlist:37i9dQZEVXcOwrS8NC07JJ', offset=0):
    sp = initialise_spotify_client()
    
    playlist_items = sp.playlist_items(playlist_id, offset=offset)
    playlist_track_meta = get_playlist_track_metadata(playlist_items)
    df_playlist = pd.DataFrame(playlist_track_meta)

    if len(playlist_items['items']) == playlist_items['limit']:
        offset += playlist_items['limit']
        df_playlist_remaining = retrieve_playlist_df(playlist_id, offset=offset)
        df_playlist = df_playlist.append(df_playlist_remaining).reset_index(drop=True)
        
    df_playlist = retrieve_track_analysis(df_playlist, track_uri_col='track_uri')
        
    return df_playlist

In [9]:
df_playlist = retrieve_playlist_df()

df_playlist.head()

 [████████████████████████████████████████████████████████████] 30/30 [00:02<00:00, 0.06s/it]

,date_added_to_playlist,album_art_url,album_art_height,album_art_width,album_uri,album_name,release_date,artist_names,artist_uris,isrc,...,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,mode,time_signature
0,2020-12-28T00:00:00Z,https://i.scdn.co/image/ab67616d0000b2731702d8...,640.0,640.0,spotify:album:5wxOkqHT4xGadN56TAzF09,Colossal Youth,1980,Young Marble Giants,spotify:artist:0tQ2Q4y9iUkKszxXyB14ZH,GBCEL0700250,...,1,-14.719,0.2130,0.944,0.810000,0.0840,0.364,166.169,1,4
1,2020-12-28T00:00:00Z,https://i.scdn.co/image/ab67616d0000b273ef277a...,640.0,640.0,spotify:album:7KQ7mXqZTuMRvQKeM3WTAw,Fargo Year 2 (Songs from the Original MGM / FX...,2016-03-25,White Denim,spotify:artist:0RdRumkn2UydUjqytNJ2Cp,USQX91600261,...,2,-8.520,0.0327,0.195,0.000198,0.1230,0.798,121.159,1,4
2,2020-12-28T00:00:00Z,https://i.scdn.co/image/ab67616d0000b27310067a...,640.0,640.0,spotify:album:2jAxcrtHoGo7SNS0r27TKk,Love is All I Bring,2019-10-25,Althea And Donna,spotify:artist:5E8FbSTegbOlYAH4b2rZj2,GBAJE7700163,...,7,-8.634,0.1620,0.164,0,0.0458,0.925,80.766,1,4
3,2020-12-28T00:00:00Z,https://i.scdn.co/image/ab67616d0000b27333c6f2...,640.0,640.0,spotify:album:3Il1CWXA64e8gukuJZoj0e,One Two,2002-03-15,Sister Nancy,spotify:artist:21pMSs2JHWwwy2kp1QIIVB,USA2P0804886,...,9,-6.813,0.2870,0.673,0.000002,0.3640,0.873,77.950,0,4
4,2020-12-28T00:00:00Z,https://i.scdn.co/image/ab67616d0000b273bc80ca...,640.0,640.0,spotify:album:63FlYaIy0oT8gYdCxBqBNe,Pickled Eggs and Sherbet,1999-01-01,The All Seeing I,spotify:artist:0A0yu52zNGKe2rtr9wHfkD,GBANR9900075,...,10,-9.136,0.0790,0.184,0.474000,0.1310,0.691,154.862,1,4


In [15]:
#hide
from nbdev.export import *
notebook2script()

Converted 01-discography.ipynb.
Converted 02-playlists.ipynb.
Converted 03-pipeline.ipynb.
